In [6]:
import sys
import os
import numpy as np
sys.path.append(os.getcwd() + '/home/hector/Documents/dynapse-se2-data/time_constant_recordings/')
#start from here
Ampa=('/AMPA')
data=np.load(data_path)



IsADirectoryError: [Errno 21] Is a directory: '/home/hector/Documents/dynapse-se2-data/time_constant_recordings//'